#JAHANZEB HASSAN

#Installing Packages  

In [1]:
!pip install accelerate -U

In [2]:
!pip install accelerate>=0.20.1

In [3]:
!pip install datasets

  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 10.0.1
    Uninstalling pyarrow-10.0.1:
      Successfully uninstalled pyarrow-10.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.


# Importing Dataset

In [4]:
from huggingface_hub import notebook_login
notebook_login()

from datasets import load_dataset
squad = load_dataset("squad", split="train[:5000]")
squad = squad.train_test_split(test_size=0.2)

import pandas as pd

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [5]:
import pandas as pd
# Convert the dataset to a dictionary
data_dict = squad["train"].to_dict()
# Create a DataFrame from the dictionary
df = pd.DataFrame.from_dict(data_dict)

# Pre-processing on Training Dataset

In [6]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

questions = [q.strip() for q in df["question"]]
context = [q.strip() for q in df["context"]]
inputs = tokenizer(
        questions,
        context,
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

offset_mapping = inputs.pop("offset_mapping")

start_positions = []
end_positions = []
answers = df['answers']
for i, offset in enumerate(offset_mapping):
    answer = answers[i]
    start_char = answer["answer_start"][0]
    end_char = answer["answer_start"][0] + len(answer["text"][0])
    sequence_ids = inputs.sequence_ids(i)

    # Find the start and end of the context
    idx = 0
    while sequence_ids[idx] != 1:
        idx += 1
    context_start = idx
    while sequence_ids[idx] == 1:
        idx += 1
    context_end = idx - 1

    # If the answer is not fully inside the context, label it (0, 0)
    if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
        start_positions.append(0)
        end_positions.append(0)
    else:
        # Otherwise it's the start and end token positions
        idx = context_start
        while idx <= context_end and offset[idx][0] <= start_char:
            idx += 1
        start_positions.append(idx - 1)

        idx = context_end
        while idx >= context_start and offset[idx][1] >= end_char:
            idx -= 1
        end_positions.append(idx + 1)

df["start_positions"] = start_positions
df["end_positions"] = end_positions

import pandas as pd
from datasets import Dataset
data = {'input_ids': inputs['input_ids'],
        'attention_mask': inputs['attention_mask'],
        'start_positions':start_positions,
        'end_positions': end_positions,
       }
df = pd.DataFrame(data)
df.to_csv('encoding_train.csv',index=False)
train = Dataset.from_pandas(df)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

#Pre-processing on Test Dataset

In [7]:
import pandas as pd
# Convert the dataset to a dictionary
data_dict = squad["test"].to_dict()
# Create a DataFrame from the dictionary
df = pd.DataFrame.from_dict(data_dict)

questions = [q.strip() for q in df["question"]]
context = [q.strip() for q in df["context"]]
inputs = tokenizer(
        questions,
        context,
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

offset_mapping = inputs.pop("offset_mapping")

start_positions = []
end_positions = []
answers = df['answers']
for i, offset in enumerate(offset_mapping):
    answer = answers[i]
    start_char = answer["answer_start"][0]
    end_char = answer["answer_start"][0] + len(answer["text"][0])
    sequence_ids = inputs.sequence_ids(i)

    # Find the start and end of the context
    idx = 0
    while sequence_ids[idx] != 1:
        idx += 1
    context_start = idx
    while sequence_ids[idx] == 1:
        idx += 1
    context_end = idx - 1

    # If the answer is not fully inside the context, label it (0, 0)
    if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
        start_positions.append(0)
        end_positions.append(0)
    else:
        # Otherwise it's the start and end token positions
        idx = context_start
        while idx <= context_end and offset[idx][0] <= start_char:
            idx += 1
        start_positions.append(idx - 1)

        idx = context_end
        while idx >= context_start and offset[idx][1] >= end_char:
            idx -= 1
        end_positions.append(idx + 1)

df["start_positions"] = start_positions
df["end_positions"] = end_positions

data = {'input_ids': inputs['input_ids'],
        'attention_mask': inputs['attention_mask'],
        'start_positions':start_positions,
        'end_positions': end_positions,
       }
df = pd.DataFrame(data)
df.to_csv('encoding_test.csv',index=False)
test = Dataset.from_pandas(df)

#Fine-tuning BERT for our dataset

In [8]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
model = AutoModelForQuestionAnswering.from_pretrained("distilbert-base-uncased")

from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

training_args = TrainingArguments(
    output_dir="BERT_QA_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=4,
    weight_decay=0.01,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train,
    eval_dataset=test,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,No log,2.390465
2,2.718600,1.777810
3,2.718600,1.678633
4,1.185600,1.692497


TrainOutput(global_step=1000, training_loss=1.9520786743164062, metrics={'train_runtime': 657.4713, 'train_samples_per_second': 24.336, 'train_steps_per_second': 1.521, 'total_flos': 1567837200384000.0, 'train_loss': 1.9520786743164062, 'epoch': 4.0})

In [10]:
tokenizer.save_pretrained('output_dir')
trainer.create_model_card()
trainer.push_to_hub()

events.out.tfevents.1708275199.ea7932f51f6d.220.0:   0%|          | 0.00/5.83k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Jahanzeb1/BERT_QA_model/commit/9ce3e3c666587a35d3f549bf4a79f873a55a0459', commit_message='End of training', commit_description='', oid='9ce3e3c666587a35d3f549bf4a79f873a55a0459', pr_url=None, pr_revision=None, pr_num=None)

In [23]:
# Evaluate the model on the test dataset
eval_results = trainer.evaluate(eval_dataset=test)

# Print evaluation results
print("Evaluation results:", eval_results)


Evaluation results: {'eval_loss': 1.6924970149993896, 'eval_runtime': 12.3852, 'eval_samples_per_second': 80.742, 'eval_steps_per_second': 5.087, 'epoch': 4.0}


# Building Pipeline

In [21]:
from transformers import pipeline


# Create a question answering pipeline
qa_pipeline = pipeline("question-answering", model='Jahanzeb1/BERT_QA_model', tokenizer='Jahanzeb1/BERT_QA_model')

# Now you can use the pipeline to answer questions
question = "How did he die?"
context = "Ernest Shackleton (15 February 1874 – 5 January 1922) led three British expeditions to the Antarctic during the Heroic Age of Antarctic Exploration. He and three companions established a new record Farthest South latitude, 112 miles (180 km) from the South Pole, as part of the Nimrod Expedition of 1907–1909, and Shackleton was knighted on his return home. He planned the Imperial Trans-Antarctic Expedition of 1914–1917 but his ship, Endurance, became trapped in pack ice and then sank on 21 November 1915. The crew escaped and used the lifeboats to reach Elephant Island and ultimately the island of South Georgia in a stormy ocean voyage of more than 700 nautical miles (800 mi; 1,300 km), Shackleton's most famous exploit. He returned to the Antarctic in 1921 with the Shackleton–Rowett Expedition, but died of a heart attack on South Georgia; at his wife's request, he was buried there. In the latter part of the 20th century, Shackleton became a role model for leadership in extreme circumstance. Stanford Question Answering Dataset (SQuAD) is a reading comprehension dataset, consisting of questions posed by crowdworkers on a set of Wikipedia articles, where the answer to every question is a segment of text, or span, from the corresponding reading passage, or the question might be unanswerable."
answer = qa_pipeline(question=question, context=context)

print("Question:", question)
print("Answer:", answer["answer"])


tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Question: How did he die?
Answer: a heart attack


#Using Gradio Interface for Our Model

In [ ]:
!pip install gradio

In [20]:
import gradio as gr

def answer_question(context, question):
    # Get answer
    answer = qa_pipeline(question=question, context=context)
    return answer['answer']

# Create Gradio interface
interface = gr.Interface(fn=answer_question,
                          inputs=["text", "text"],
                          outputs="text",
                          title="Question Answering with BERT",
                          description="Enter a context and a question to get the answer.",
                          )
interface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://e12c648f2ca21ca9c1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


# Link to the deployed app

In [ ]:
https://huggingface.co/spaces/Jahanzeb1/BERT-QA